In [2]:
!pip install boto3

In [21]:
#連線 dynamodb 
#引用套件
import boto3
from boto3.dynamodb.conditions import Key, Attr
# Get the service resource.
dynamodb = boto3.resource(

    'dynamodb',

    endpoint_url='http://cc104.dynamodb.local:8000',   #連線網址192.168.254.100:8000 虛機ip位址，在虛機內開8

    region_name='my_region',                           #區域名稱 選用虛擬區域

    aws_access_key_id='my_access_key',                 #aws進入鑰匙id選用虛擬進入key

    aws_secret_access_key='my_secret_key',             #aws私鑰選用虛擬私鑰進去

    verify=False)

In [32]:
# Create the DynamoDB table.
table = dynamodb.create_table(
    TableName='users',
    KeySchema=[
        {
            'AttributeName': 'username',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'last_name',
            'KeyType': 'RANGE'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'username',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'last_name',
            'AttributeType': 'S'
        },

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

# Wait until the table exists.
table.meta.client.get_waiter('table_exists').wait(TableName='users')

# Print out some data about the table.
print(table.item_count)

0


In [35]:
#插入資料
table.put_item(
   Item={
        'username': 'janedoe',
        'first_name': 'Jane',
        'last_name': 'Doe2',
        'age': 2521,
        'account_type': 'standard_user2',
    }
)

print('afterput:', table.item_count)


afterput: 2


In [20]:
#查詢資料
response = table.get_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    }
)
item = response['Item']
print(item)

{'last_name': 'Doe', 'account_type': 'standard_user', 'first_name': 'Jane', 'age': Decimal('25'), 'username': 'janedoe'}


In [15]:
#更新物件
table.update_item(
    Key={
        'username': 'janedoe2',
        'last_name': 'Doe2'
    },
    UpdateExpression='SET age = :val1',
    ExpressionAttributeValues={
        ':val1': 111111111
    }
)

{'ResponseMetadata': {'RequestId': '3474f352-67cf-472a-b472-55280fe331ce',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2745614147',
   'x-amzn-requestid': '3474f352-67cf-472a-b472-55280fe331ce',
   'content-length': '2',
   'server': 'Jetty(8.1.12.v20130726)'},
  'RetryAttempts': 0}}

In [17]:
#刪除物件
table.delete_item(
    Key={
        'username': 'janedoe2',
        'last_name': 'Doe2'
    }
)

{'ResponseMetadata': {'RequestId': '7342595c-569f-42d1-a4b5-d0edf4648692',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '2745614147',
   'x-amzn-requestid': '7342595c-569f-42d1-a4b5-d0edf4648692',
   'content-length': '2',
   'server': 'Jetty(8.1.12.v20130726)'},
  'RetryAttempts': 0}}

In [36]:
#查詢
response = table.query(
    KeyConditionExpression=Key('username').eq('janedoe')
)
items = response['Items']
print(items)

[{'last_name': 'Doe', 'account_type': 'standard_user2', 'first_name': 'Jane', 'age': Decimal('2521'), 'username': 'janedoe'}, {'last_name': 'Doe2', 'account_type': 'standard_user2', 'first_name': 'Jane', 'age': Decimal('2521'), 'username': 'janedoe'}]


In [54]:
#掃描
response = table.scan(
    FilterExpression=Attr('age').lt(5000)
)
items = response['Items']
print(items)

[{'last_name': 'Doe', 'account_type': 'standard_user2', 'first_name': 'Jane', 'age': Decimal('2521'), 'username': 'janedoe'}, {'last_name': 'Doe2', 'account_type': 'standard_user2', 'first_name': 'Jane', 'age': Decimal('2521'), 'username': 'janedoe'}]


In [30]:
#刪除table
table.delete()

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'username',
    'AttributeType': 'S'},
   {'AttributeName': 'last_name', 'AttributeType': 'S'}],
  'TableName': 'users',
  'KeySchema': [{'AttributeName': 'username', 'KeyType': 'HASH'},
   {'AttributeName': 'last_name', 'KeyType': 'RANGE'}],
  'TableStatus': 'ACTIVE',
  'CreationDateTime': datetime.datetime(2019, 1, 17, 11, 26, 26, 99000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'LastIncreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'LastDecreaseDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 5,
   'WriteCapacityUnits': 5},
  'TableSizeBytes': 295,
  'ItemCount': 4,
  'TableArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/users'},
 'ResponseMetadata': {'RequestId': 'ca439e0b-0f17-456c-8111-1a3e99a474af',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '39